#Продвинутый Python, ДЗ-4

Правила игры:

В домашке 6 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью plotly или же Dash. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [1]:
# !pip install wget

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

# Часть 1. Plotly

В данной части для заданий нужно использовать Plotly для того, чтобы отобразить графики

In [22]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from dateutil.parser import parse
from datetime import datetime

## Задание 1 (15 баллов)

Отобразите на карте всех продавцов (в качестве точек - координаты, в качестве размера и цвета точек - число продавцов). При наведении точек должно отображаться название штата, города, а также число продавцов

*Hint: внимательно посмотрите на датасет locations. Если делать задачу максимально втупую, то у вас либо крашнется/зависнет отрисовка, либо на это нельзя будет нормально смотреть*

In [36]:
unique_locations = location.drop_duplicates(subset=['geolocation_city'])
sellers_tmp = sellers.copy()
sellers_tmp.rename(columns={'seller_city': 'geolocation_city'}, inplace = True)
res = sellers_tmp.join(unique_locations.set_index('geolocation_city'), how='inner', on='geolocation_city')
res = res.groupby('geolocation_city').agg({'seller_id' : 'nunique'}).reset_index()
res = res.join(unique_locations.set_index('geolocation_city'), how='inner', on='geolocation_city')
res['size'] = res.seller_id.apply(lambda x: np.log2(x + 1) * 2)
res

,geolocation_city,seller_id,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,size
0,abadia de goias,1,75345,-16.766457,-49.434210,GO,2.000000
1,afonso claudio,1,29600,-20.072235,-41.132607,ES,2.000000
2,alambari,1,18220,-23.568093,-47.889391,SP,2.000000
3,alfenas,2,37137,-21.407289,-45.940358,MG,3.169925
4,almirante tamandare,1,83507,-25.316851,-49.304592,PR,2.000000
...,...,...,...,...,...,...,...
541,volta redonda,2,27265,-22.529850,-44.124115,RJ,3.169925
542,votorantim,4,18117,-23.571057,-47.457454,SP,4.643856
543,votuporanga,4,15503,-20.399208,-49.969640,SP,4.643856
544,xanxere,2,89820,-26.870625,-52.404488,SC,3.169925


In [37]:
fig = go.Figure(go.Scattermapbox(lat=res['geolocation_lat'],
                                 lon=res['geolocation_lng'],
                                 text=[f"Город: {Town}, Штат: {State}, Количество продавцов: {Quantity}" for Town, State, Quantity in zip(res['geolocation_city'], res['geolocation_state'], res['seller_id'])],


                                 marker=dict(colorbar=dict(title="Количество продавцов"),
                                             color=res['seller_id'],
                                             size=res['size'])
                                 ))
fig.update_layout(mapbox_style="open-street-map",
                  title={
                     "text": "Распределение продавцов Бразилии",
                      "x": 0.5
    })
fig.show()

## Задание 2 (15 баллов)

Отобразите на графике число заказов по дням с разбивкой по статусам. Также создайте график со средней оценкой отзыва (аггрегируйте по дням). Для обоих графиков должен быть слайдер по датам

In [38]:
orders_copy = orders.copy()
orders_copy['order_purchase_timestamp'] = pd.to_datetime(orders_copy['order_purchase_timestamp']).dt.date
res = orders_copy.join(reviews.set_index('order_id'), how='inner', on='order_id')
status = np.array(orders.order_status.unique())
result_dict = {}
for i in status:
    result_dict[i] = [orders_copy[orders_copy['order_status'] == i].groupby('order_purchase_timestamp').agg({'order_id': 'nunique'}).reset_index(), 
               res[res['order_status'] == i].groupby('order_purchase_timestamp').agg({'review_score':'mean'}).reset_index()]

In [39]:
fig = go.Figure()
fig = make_subplots(rows=1, cols=2, subplot_titles=["Число заказов по дням", "Средняя оценка отзыва"])
for key in result_dict:
    fig.add_trace(go.Scatter(x=result_dict[key][0].order_purchase_timestamp, y=result_dict[key][0].order_id, visible=True, opacity=0.4, 
                            text = [f"Дата заказа: {c.year}-{c.month}-{c.day}, Количество заказов: {s}" for c, s,in zip(result_dict[key][0]['order_purchase_timestamp'],result_dict[key][0]['order_id'])], 
                            name=i,mode='markers', legendgroup=i), row = 1, col = 1)
    fig.add_trace(go.Scatter(x=result_dict[key][1].order_purchase_timestamp, y=result_dict[key][1].review_score, visible=True, opacity=0.4, 
                             text = [f"Дата заказа: {c.year}-{c.month}-{c.day}, Средняя оценка: {s}" for c, s,in zip(result_dict[key][1]['order_purchase_timestamp'], result_dict[key][1]['review_score'])], 
                             name=i,mode='markers', legendgroup=i), row = 1, col = 2)

fig.update_layout(
    showlegend=True,
     legend={
        "title": "Cтатус заказа"
    })
fig.update_xaxes(
    dict(
        rangeslider=dict(
            visible=True
        ),
        type="date",
    ),
    row = 1,
    col = 1,
    title="Дата",
)
fig.update_xaxes(
    dict(
        rangeslider=dict(
            visible=True
        ),
        type="date",
    ),
    row = 1,
    col = 2,
    title="Дата",
)
fig.update_yaxes(title_text="Число заказов", row=1, col=1)
fig.update_yaxes(title_text="Средняя оценка", row=1, col=2)

fig.show()

## Задание 3 (20 баллов)

Выберите топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [34]:
df = orders[['order_id', 'customer_id', 'order_status']].join(items[['order_id', 'seller_id']].set_index('order_id'), how='inner', on='order_id')
df = df.join(sellers[['seller_id', 'seller_state']].set_index('seller_id'), on='seller_id', how='inner')
df = df.join(customers[['customer_id', 'customer_state']].set_index('customer_id'), on='customer_id', how='inner')# склеил датафреймы
df = df[df.order_status == 'delivered'] # убрал не доставленные
df = df.groupby(['seller_id', 'seller_state', 'customer_state']).agg({'order_id': 'nunique'}).reset_index()

df_cnt_orders = df.copy()
df_cnt_orders = df_cnt_orders.groupby(['seller_id']).agg({'order_id': 'sum'}).reset_index()
df_cnt_orders = df_cnt_orders[df_cnt_orders.order_id > 20] # отобрал те где больше 100 продаж

df = df[df.seller_state != df.customer_state]
df = df.groupby(['seller_id']).agg({'order_id': 'sum'}).reset_index()
df = df.rename(columns={'order_id': 'another_state'}) # нашёл их заказы в другие регионы

res = df_cnt_orders.join(df.set_index('seller_id'), how='inner', on='seller_id')
res['share'] = res['another_state'] / res['order_id']
res = res.sort_values('share', ascending=False)
res = res.head(5)
res = res[['seller_id']]


res = res.join(sellers.set_index('seller_id'), how='inner', on='seller_id')
res = pd.merge(location, res, left_on='geolocation_zip_code_prefix',right_on= 'seller_zip_code_prefix')
res = res.groupby(by=['seller_id']).agg({'geolocation_lat':'mean', 'geolocation_lng':'mean'}).reset_index()
res = res.rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'})
res = res.join(items.set_index('seller_id'), how='inner', on='seller_id')
res = res.join(orders.set_index('order_id'), how='inner', on='order_id')
res = res.join(customers.set_index('customer_id'), how='inner', on='customer_id')
res = pd.merge(location, res, left_on='geolocation_zip_code_prefix',right_on= 'customer_zip_code_prefix')
res = res.groupby(by=['customer_id', 'seller_id', 'seller_lat', 'seller_lng']).agg({'geolocation_lat':'mean', 'geolocation_lng':'mean'}).reset_index()


data_to_draw = []
for index, row in res.iterrows():
    data_to_draw.append(
        go.Scattermapbox(
            lat=[row['seller_lat'], row['geolocation_lat']],
            lon=[row['seller_lng'], row['geolocation_lng']],
            mode='lines+markers',
            text='От кого | откуда: ' + ' | '  +  '<br>' +  'Кому | куда: '  + ' | ' 
        )
    )

fig3 = go.Figure(data=data_to_draw)
fig3.update_layout(mapbox_style="carto-darkmatter",
    title = {
        "text": "5 Лучших продавцов",
        "x": 0.5
    },
    mapbox=dict(
        center=dict(
            lat=-25, 
            lon=-45 
        ),
        zoom=2.5,
    ),

    paper_bgcolor='LightCoral',
    showlegend=False
     
)
fig3.show()
# print(data_to_draw)


# Часть 2. Dash

В данной части нужно сделать и сдать один .py файл (можно назвать dash_task.py), который будет разворачивать страничку с нарисованными графиками (с помощью Dash)

## Задание 4 (20 баллов)

Сделайте сайт, где будет отображаться следующее:

* График, показывающий распределение по категориям продаж (то есть по тому, что продают селлеры) в штате

* График, показывающий распределение по категориям покупок (то есть по тому, что покупают пользователи) в штате

* Фильтр по штатам (должна быть возможность как выбрать отдельный штат, так и вся Бразилия в целом)

* Возможность выбрать, какие статусы продаж учитывать (чтобы можно было выбрать несколько статусов)

* Слайдер по датам (чтобы можно было выбрать даты, по которым можем смотреть результаты)


## Задание 5 (15 баллов)

Добавить карту, на котором можно будет посмотреть на количество продавцов или на количество покупателей (на него также должны работать фильтры по датам и по статусам, реализованные в прошлом занятии). Аггрегацию нужно делать по штатам. Чтобы вам помочь, вот snippet кода, который можно использовать для отображения:

In [ ]:
import json
import pandas as pd
import plotly as plt
import plotly.express as px
from urllib.request import urlopen

# Brazil coordinates / shape
with urlopen(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
) as response:
    brazil = json.load(response)

# Since the database doesn't have an ID or feature using which values will be mapped between the coordinate/shape database and soybean database, we are adding an ID ourselves.
for feature in brazil["features"]:
    feature["id"] = feature["properties"]["sigla"]

# Здесь в качестве id, который будет отображаться, указано сокращение названия штата (как и в наших данных)

*Hint: обратите внимание, что покупатели и продавцы могут быть не во всех штатах, а эжто значит, что они могут не отображаться на карте. Эту ситуацию нужно обрабатывать и поправить*

## Задание 6 (15 баллов)

Сделайте карту кликабельной: по клику на штат должен показываться сам штат (и только он), все остальные графики обновляются по выбранному штату